# Embeddings

[Embedding 모델 leaderboard] https://huggingface.co/spaces/mteb/leaderboard

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from openai import OpenAI

client = OpenAI()
text = "좋은 아침"

response = client.embeddings.create(
    model='text-embedding-3-small',
    input=[text]
)

In [3]:
print(response.data[0].embedding)
print(len(response.data[0].embedding))

[-0.005385647993534803, 0.013474476523697376, -0.09155601263046265, -0.021604733541607857, 0.022329725325107574, -0.05551360175013542, 0.04631657153367996, 0.011827711015939713, 0.02583039551973343, -0.03314244747161865, -0.017865851521492004, 0.0030604980420321226, -0.013401977717876434, 0.013950899243354797, 0.001657122396863997, 0.02491897903382778, -0.04184234142303467, 0.014924459159374237, 0.02317900024354458, 0.03838310018181801, -0.0058620707131922245, -0.035773131996393204, -0.005587609484791756, 0.03163032606244087, 0.011475573293864727, 0.03129889816045761, 0.006058854050934315, 0.0012065923074260354, 0.023282570764422417, -0.044162314385175705, -0.0046865493059158325, -0.04308518394827843, 0.008446145802736282, -0.013785187155008316, 0.031526755541563034, 0.025975394994020462, 0.0032080854289233685, -0.02786037139594555, -0.06872915476560593, -0.030180342495441437, 0.005222524981945753, 0.016736935824155807, 0.033287446945905685, -0.04383089020848274, 0.03714025765657425, -

### 음식 리뷰 데이터 활용

In [4]:
import pandas as pd

df = pd.read_csv('fine_food_reviews_1k.csv')
df

,Unnamed: 0,Time,ProductId,UserId,Score,Summary,Text
0,0,1351123200,B003XPF9BO,A3R7JR3FMEBXQB,5,where does one start...and stop... with a tre...,Wanted to save some to bring to my Chicago fam...
1,1,1351123200,B003JK537S,A3JBPC3WFUT5ZP,1,Arrived in pieces,"Not pleased at all. When I opened the box, mos..."
2,2,1351123200,B000JMBE7M,AQX1N6A51QOKG,4,"It isn't blanc mange, but isn't bad . . .",I'm not sure that custard is really custard wi...
3,3,1351123200,B004AHGBX4,A2UY46X0OSNVUQ,3,These also have SALT and it's not sea salt.,I like the fact that you can see what you're g...
4,4,1351123200,B001BORBHO,A1AFOYZ9HSM2CZ,5,Happy with the product,My dog was suffering with itchy skin. He had ...
...,...,...,...,...,...,...,...
995,995,1351209600,B004OQLIHK,AKHQMSUORSA91,5,Delicious!,I have ordered these raisins multiple times. ...
996,996,1351209600,B0006349W6,A21BT40VZCCYT4,5,Good Training Treat,My dog will come in from outside when I am tra...
997,997,1351209600,B00611F084,A6D4ND3C3BCYV,5,Jamica Me Crazy Coffee,Wolfgang Puck's Jamaica Me Crazy is that wonde...
998,998,1351209600,B005QKH5HA,A3LR9HCV3D96I3,5,Party Peanuts,Great product for the price. Mix with the Asia...


In [5]:
# 토큰 계산
import tiktoken

# gpt4o_encoding = tiktoken.encoding_for_model('gpt-4o')
gpt4o_encoding = tiktoken.encoding_for_model('text-embedding-3-small')

df['n_tokens'] = df['Text'].apply(lambda x: len(gpt4o_encoding.encode(x)))
df[['Text', 'n_tokens']].head()

,Text,n_tokens
0,Wanted to save some to bring to my Chicago fam...,34
1,"Not pleased at all. When I opened the box, mos...",26
2,I'm not sure that custard is really custard wi...,249
3,I like the fact that you can see what you're g...,223
4,My dog was suffering with itchy skin. He had ...,77


In [6]:
df['n_tokens'].describe()

count    1000.000000
mean       85.637000
std        73.562903
min        22.000000
25%        38.000000
50%        60.000000
75%       106.500000
max       623.000000
Name: n_tokens, dtype: float64

In [7]:
# 임베딩 처리
def texts_to_embedding(texts):

    # 전처리 - 불용어 등
    texts = [text.replace('\n', ' ') for text in texts]

    response = client.embeddings.create(
        model='text-embedding-3-small',
        input=texts
    )

    return [data.embedding for data in response.data]

In [8]:
df['embedding'] = texts_to_embedding(df['Text'].tolist())

In [9]:
df['embedding']

0      [0.016770517453551292, -0.008579619228839874, ...
1      [-0.005239809397608042, 0.0404508151113987, -0...
2      [0.005527602043002844, -0.012969312258064747, ...
3      [-0.016285082325339317, 0.008895758539438248, ...
4      [-0.004306328017264605, -0.06373810768127441, ...
                             ...                        
995    [0.01908072456717491, -0.03722948580980301, -0...
996    [-0.037062328308820724, -0.01365215890109539, ...
997    [-0.04671481251716614, -0.07131096720695496, -...
998    [-0.014545817859470844, -0.03374110162258148, ...
999    [0.014057457447052002, -0.013706959784030914, ...
Name: embedding, Length: 1000, dtype: object

In [10]:
embed_df = df['embedding'].to_frame('embedding')
embed_df.index = df['Text']
embed_df

,embedding
Text,
Wanted to save some to bring to my Chicago family but my North Carolina family ate all 4 boxes before I could pack. These are excellent...could serve to anyone,"[0.016770517453551292, -0.008579619228839874, ..."
"Not pleased at all. When I opened the box, most of the rings were broken in pieces. A total waste of money.","[-0.005239809397608042, 0.0404508151113987, -0..."
"I'm not sure that custard is really custard without eggs. But this comes close. I got it for use in a ""Vegan pancake"" recipe. We were having houseguests who were Vegan and I wanted to make some special breakfasts while they were here. One of the cooking/recipe sites had a recipe using this and there were lots of great reviews. I tried the recipe and it turned out like wallpaper paste -- yuck!<br />However, the so-called custard isn't so bad. I think it's probably just cornstarch and annatto (yellow coloring with a slight flavor). It's fun playing with it. You could dress it up with fruit. Seems to come out on the thin side when you make it as directed, so I use less milk because I like my custards to set firm. As a custard sauce it's fine. I would say it tastes something between a pudding and a custard.<br /><br />If you want a really good egg-free ""custard"" get an original recipe for ""blanc mange."" It takes a lot longer to make, but it's certainly worth the difference.","[0.005527602043002844, -0.012969312258064747, ..."
"I like the fact that you can see what you're getting and that there are no bones or dark meat. There are 7 nice big chunks in every jar.<br /><br />These taste like tuna in a can but, because they're preserved in glass, you don't have to worry about either aluminum or BPA; BUT ... they are not just tuna and spring water.<br /><br />There is salt in there, too, and it's not healthy sea salt, it's toxic table salt.<br /><br />I am trying to contact Tonnino to confirm that. I might be wrong because the label states that the ingredients are ""tuna fish"" but the sticker on the top clarifies that it is the smaller (healthier) yellowfin, so the ""salt"" listed in the ingredients might be sea salt but, if it was, why don't they say so?<br /><br />Without confirmation, I will continue to look for a salt-free olive-oil free tuna preserved in glass.<br /><br />If you know of one, please contact me!","[-0.016285082325339317, 0.008895758539438248, ..."
My dog was suffering with itchy skin. He had been eating Natural Choice brand (cheaper) since he was a puppy. I was nervous to change foods. The vet suggested to change foods sand see if the skin issues cleared up. Wellness brand did the job. My dog seems to love the food and the skin issues cleared up within a few weeks.,"[-0.004306328017264605, -0.06373810768127441, ..."
...,...
I have ordered these raisins multiple times. They are always great and arrive timely. I can't go back to store bought chocolate covered raisins now! Love this product.,"[0.01908072456717491, -0.03722948580980301, -0..."
My dog will come in from outside when I am training her and look at the cupboard waiting for her treat. When I use the clicker training method she comes because she knows she has something special.,"[-0.037062328308820724, -0.01365215890109539, ..."
Wolfgang Puck's Jamaica Me Crazy is that wonderful blend of island flavors in a coffee. Have loved it from the first time tasting. Great product.,"[-0.04671481251716614, -0.07131096720695496, -..."


- 코사인 유사도 측정

In [11]:
import numpy as np

cos_sim = lambda a, b: np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_similar_texts(query, embed_df, top_n=5):
    query_embed = texts_to_embedding(query)[0]
    embed_df['cos_sim'] = embed_df['embedding'].apply(lambda x: cos_sim(x, query_embed))
    return embed_df.sort_values('cos_sim', ascending=False).head(top_n)

In [12]:
get_similar_texts(['coffee'], embed_df)

,embedding,cos_sim
Text,,
"I I haven't had a bad cup of coffee yet. So far, my favorites are the Decaf, Columbian, and Breakfast blend. I only drink one cup of coffee a day, so this type of coffee maker is perfect for me. Especially since I only like coffee when it's hot and fresh. My guests love it too!","[-0.01845250464975834, -0.05068936571478844, 0...",0.419543
"I have a coffee maker that grinds my coffee beans. It's hard to find whole bean decafinated coffee. When I find it in the brand that I like, I am excited. Seattle's Best is my favorite.","[-0.03934145346283913, -0.03586488589644432, -...",0.406183
So yummy... Drinking it Black coffee or w cream this coffee is delish .... One of my favs... I would recommend this coffee to everyone :),"[0.01336692739278078, -0.04671807214617729, -0...",0.402114
So yummy... Drinking it Black coffee or w cream this coffee is delish .... One of my favs... I would recommend this coffee to everyone :),"[0.01336692739278078, -0.04671807214617729, -0...",0.402114
"So my wife is a latte freak, and nursing, so decaf is the approved type. After the Senseo left the market, I struggled and found the <a href=""http://www.amazon.com/gp/product/B0047BIWSK"">Aerobie AeroPress Coffee and Espresso Maker</a> which is like a French Press for the 21st century. After getting our recipe figured out, my wife, who's been buying Venti Decaf Latte's at $4 a pop almost daily for years now declares that Seattle's best Level 3 Decaf in her home-made Latte is the best coffee she can get. We've tried other bands, and this is her favorite, hands down!","[-0.013580745086073875, -0.04869876429438591, ...",0.400505
